In [1]:
%cd  /data/PyDeepLearning/END2END/MSMatch

/data/PyDeepLearning/END2END/MSMatch


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"


# import needed library
import os
import logging
import random
import warnings

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp

from utils import net_builder, get_logger, count_parameters, create_dir_str
from train_utils import TBLog, get_OPT, get_cosine_schedule_with_warmup
from models.fixmatch.fixmatch import FixMatch
from datasets.ssl_dataset import SSL_Dataset
from datasets.data_utils import get_data_loader

/home/warmachine/anaconda3/envs/msmatch_cu11/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Construct Dataset
train_dset = SSL_Dataset(name='thraws_swir', train=True, data_dir='/data/PyDeepLearning/END2END/MSMatch/DATA/warmup_events_dataset', seed=42, )
lb_dset, ulb_dset = train_dset.get_ssl_dset(6)

100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


In [4]:
lb_dset.data.shape

(6, 256, 256, 3)

In [7]:
train_dset.get_data()[0].shape

100%|██████████| 2/2 [00:01<00:00,  1.08it/s]


(4075, 256, 256, 3)

In [8]:
from pathlib import Path 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


datadir = Path('/data/PyDeepLearning/END2END/MSMatch/DATA/warmup_events_dataset')
event_dir = datadir / 'event'
notevent_dir = datadir / 'notevent'

events = [x for x in event_dir.iterdir()]
notevents = [x for x in notevent_dir.iterdir()]

filenames = [x.as_posix() for x in events+notevents]
labels = [1 for x in events]+[0 for x in notevents]

In [41]:
size = 256
labels = np.asarray(labels)
filenames = np.asarray(filenames)
N = len(filenames)

images = np.zeros([N, size, size, 3], dtype="uint8")

# convert to integer labels
le = preprocessing.LabelEncoder()
le.fit(np.sort(np.unique(labels)))
labels = le.transform(labels)
labels = np.asarray(labels)

# split into a train and test set as provided data is not presplit
X_train, X_test, y_train, y_test = train_test_split(
                                          images,
                                          labels,
                                          test_size=0.5,
                                          random_state=42,
                                          stratify=labels,
                                          )


In [42]:
X_train

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ...,

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0,

In [44]:
y_train

array([0, 0, 1, ..., 0, 0, 0])

In [62]:
def upsample_ds(ds, lb, N, M, tol, recursive=True):
       X_train_new, y_train_new = [], []
       total_events = lb.sum()
       for img, label in zip(ds, lb):
              if label == 0:
                     for _ in range(N):
                            X_train_new.append(img)
                            y_train_new.append(label)
              else:
                     for _ in range(M):
                            X_train_new.append(img)
                            y_train_new.append(label)

       X_train_new=np.asarray(X_train_new, dtype='uint8')
       y_train_new=np.asarray(y_train_new)


       if recursive:
              total_events = y_train_new.sum()
              print('Shape: ',X_train_new.shape[0] // 2)
              print('total_events: ',total_events)
              if not (abs(X_train_new.shape[0] // 2 - total_events) < tol):
                     X_train_new, y_train_new = upsample_ds(ds=ds, lb=lb, N=N, M=M+1, tol=100)

       return X_train_new, y_train_new   

In [63]:
N, M = 1, 3

X_train_ups, y_train_ups = upsample_ds(ds=X_train, lb=y_train, N=N, M=M, tol=100)

Shape:  1310
total_events:  534
Shape:  1399
total_events:  712
Shape:  1488
total_events:  890
Shape:  1577
total_events:  1068
Shape:  1666
total_events:  1246
Shape:  1755
total_events:  1424
Shape:  1844
total_events:  1602
Shape:  1933
total_events:  1780
Shape:  2022
total_events:  1958


In [31]:
y_train_new.sum()

1424